In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Algorithmen

In [ ]:
%run ./Muehle_Logic.ipynb
%run ./Muehle_Heuristik.ipynb

In [ ]:
Cache_Memoize = {}

In [ ]:
def memoize(f):
    global Cache_Memoize
    
    def f_memoized(*args):
        key = (args[0], args[1], args[2])
        if key in Cache_Memoize:
            return Cache_Memoize[key]
        result = f(*args)
        Cache_Memoize[key] = result
        return result
    
    return f_memoized

## Minimax-Algorithmus

Der Minimax-Algorithmus wird im Rahmen dieser Studienarbeit für die Ermittlung der optimalen Strategie für das Nullsummenspiel Mühle verwendet. Bei einem Nullsummenspiel wie Mühle ist der Gewinn eines Spielers gleichzeitig der Verlust des anderen Spielers. Die Summe ist somit null. Der Minimax-Algorithmus ist hierbei ein relativ einfacher Algorithmus für Nullsummenspiele, der durch die spätere Verwendung von Alpha-Beta-Pruning deutlich verbessert werden kann.

Der Minimax-Algorithmus beruht auf einer Bewertungsfunktion, der Heuristik und systematischer Suche. Grundlegend werden bei Minimax alle auf den aktuellen Spielstatus folgenden Zustände berechnet und bewertet. Dies ist vergleichbar mit einer Baumstruktur, bei der bis zu den Blättern alle Zustände ausgewertet werden. Da dies aus Gründen der Rechenzeit und des Speichers nicht möglich ist, werden die Folgezustände nur bis zu einer gewissen Tiefe berechnet und ausgewertet. Um nur bis zu einer geringen Baumtiefe suchen zu müssen wird allerdings eine geeignete Heuristik benötigt, da wir mit der übergebenen Suchtiefe nicht immer einen Endzustand erreichen und den Zustand somit nicht eindeutig bewerten können. Mit der Nutzung dieser Heuristik verlieren wir also die Sicherheit den optimalen Zug zu wählen, erhalten aber immerhin eine gute Einschätzung und eine akzeptable Rechenzeit. Ohne eine Heuristik müssten wir alle Spielverläufe im Vorfeld ausrechnen, was bei nicht trivialen Nullsummenspielen äußerst aufwendig, beziehungsweise nicht möglich ist.

Minimax-Baumstruktur-Beispiel:
Die Werte innerhalb der Knoten/Blätter entsprechen der Bewertung des jeweiligen Zustandes

<img src="minimax.png" width="800"/>

Quelle: [https://stackabuse.com/minimax-and-alpha-beta-pruning-in-python/]





In [ ]:
import random
random.seed(1)

Die Funktion `value_minimax(state, player, depth)` erhält drei Argumente. Einen Spielzustand, einen Spieler und die Suchtiefe. Die Funktion gibt dabei den Wert zurück, den der übergebene Spielzustand für den übergeben Spieler hat.
Dieser Wert wird, für den Fall, dass das Spiel mit dem übergebenen Zustand beendet ist, von der Funktion `utility()` berechnet. Wenn die maximale Suchtiefe erreicht wurde wird der Wert allerdings von der Funktion `heuristic()` berechnet.
Ist die maximale Suchtiefe noch nicht erreicht wird rekursiv nach dem besten Folgezustand gesucht.
Um Werte für Zustände nicht mehrfach berechnen zu müssen werden diese durch Memoisation `@memoize` zwischengespeichert.


In [ ]:
@memoize
def value_minimax(state, player, depth):
    if finished(to_list(state), player):
        return utility(to_list(state), player)
    if depth == 0:
        return heuristic(state, player)
    o = opponent(player)
    depth -= 1
    return max([ -value_minimax(to_tuple(ns), o, depth) for ns in next_states(to_list(state), player) ])

Die Funktion `best_move_minimax(state, player, depth)` erhält drei Argumente. Einen Spielzustand, einen Spieler und die Suchtiefe. Die Rückgabewerte sind, der von der Funktion ermittelte, beste Folgezustand und dessen Bewertung. Gibt es mehrere beste Folgezustände wird der Folgezustand zufällig ausgewählt.

Die besten Züge werden von der Funktion durch das wiederholte negierte Aufrufen der Funktion `value_minimax()` bestimmt, da für den Folgezustand mit dem Gegner als Spieler gesucht werden muss. Genauer gesagt müssen wir, da wir uns in einem Nullsummenspiel befinden, die Bewertungen für alle berechneten Züge negieren, um für den, der Funktion `best_move_minimax()`, übergebenen Spieler den besten Zug zu wählen.

In [ ]:
def best_move_minimax(state, player, depth):
    ns          = next_states(state, player)
    best_value  = value_minimax(to_tuple(state), player, depth)
    best_moves  = [s for s in ns if -value_minimax(to_tuple(s), opponent(player), depth - 1 ) == best_value]
    best_state  = random.choice(best_moves)
    return best_value, best_state

Die Funktion `minimax(state, player, depth)` wurde erstellt, um die Funktion `best_move_minimax()` nach außen hin eindeutiger von `best_move_ab()` abzugrenzen. Die übergebenen Argumente und Rückgabewerte entsprechen somit denen der Funktion `best_move_minimax()`.

In [ ]:
def minimax(state, player, depth = 5):
    return(best_move_minimax(state, player, depth))

## Alpha-Beta-Pruning
Das Alpha-Beta-Pruning ist, wie im Rahmen des Minimax-Algorithmus schon erwähnt eine Verbesserung von Minimax. Beim Alpha-Beta-Pruning wird die Auswertung von Teilbäumen abgebrochen, sobald klar ist, dass keine Verbesserung erwartbar ist. Durch diese Technik wird die Rechenzeit bei steigender Suchtiefe gegenüber Minimax erheblich reduziert. 
Das Hauptkonzept von Alpha-Beta-Pruning besteht darin, die Werte Alpha und Beta über die gesamte Suche hinweg mitzunehmen. Alpha enthält dabei den bestmöglichen Wert der erkundeten Optionen für den maximierenden Spieler und Beta das gleiche für den minimierenden Spieler, wobei Alpha und Beta initial auf dem für Alpha und Beta schlechtesten Wert starten. (Alpha = -1, Beta = 1)

In [ ]:
Cache_AB = {}

In [ ]:
#Cache = {}

Der Funktion `value_ab(state, player, alpha=-1, beta=1)` werden vier Argumente übergeben. Einen Spielzustand, einen Spieler, Alpha, Beta und die Suchtiefe. Dabei gibt value_ab wie value_minimax, den ermittelten Wert für den übergebenen Spielzustand und den übergebenen Spieler zurück.

In [ ]:
def value_ab(state, player, alpha=-1, beta=1, depth = 6):
    global Cache_AB
    state = to_tuple(state)
    if (state, player, depth) in Cache_AB:
        value, a, b = Cache_AB[(state, player, depth)]
        if a <= alpha and beta <= b:
            return value
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            value   = alpha_beta(state, player, alpha, beta, depth)
            Cache_AB[(state, player, depth)] = value, alpha, beta
            return value
    else:
        value = alpha_beta(state, player, alpha, beta, depth)
        Cache_AB[(state, player, depth)] = value, alpha, beta
        return value

In [ ]:
#def value_ab(state, player, alpha=-1, beta=1, depth = 6):
#    global Cache
#    state = to_tuple(state)
#    if state in Cache:
#        value, a, b = Cache[state]
#        if a <= alpha and beta <= b:
#            return value
#        else:
#            alpha = min(alpha, a)
#            beta  = max(beta , b)
#            value   = alpha_beta(state, player, alpha, beta, depth)
#            Cache[state] = value, alpha, beta
#            return value
#    else:
#        value = alpha_beta(state, player, alpha, beta, depth)
#        Cache[state] = value, alpha, beta
#        return value

Die Funktion `alpha_beta(state, player, alpha, beta)` erhält die gleichen Werte wie die Funktion `value_ab()` und gibt den Wert für den übergebenen Zustand zurück. Dabei arbeitet `alpha_beta()` nach den im Skript vorgestellten Regeln:
- $\alpha \leq \texttt{value_ab}(state, player) \leq \beta \;\rightarrow\;\texttt{alpha_beta}(state, player, \alpha, \beta) = \texttt{value}(state,player)$
- $\texttt{value}(state, player) < \alpha \;\rightarrow\; \texttt{alpha_beta}(state, player, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{value}(state, player) \;\rightarrow\; \beta \leq \texttt{alpha_beta}(state, player, \alpha, \beta)$

In [ ]:
def alpha_beta(state, player, alpha, beta, depth):
    state = to_list(state)
    if finished(state, player):
        return utility(state, player)
    if depth == 0:
        return heuristic(state, player)
    value = alpha
    ns_value = []
    if depth > 4:
        for ns in next_states(state, player):
            ns_value += [(ns, -heuristic(state, opponent(player)))] #testweise - entfernen oder anders herum sortieren
            ns_value.sort(key=lambda x: x[1])
        for ns_v in ns_value:
            ns = ns_v[0]
            value = max(value, -value_ab(ns, opponent(player), -beta, -alpha, depth-1))
            if value >= beta:
                return value
            alpha = max(value, alpha)
    else:
        for ns in next_states(state, player):
            value = max(value, -value_ab(ns, opponent(player), -beta, -alpha, depth-1))
            if value >= beta:
                return value
            alpha = max(value, alpha)
    return value

In [ ]:
ns_value = [('test',2), ('abcd',1)]
ns_value.sort(key=lambda x: x[1])
ns_value

Die Funktion `best_move_ab(state, player, depth = 5)` erhält drei Argumente. Einen Spielzustand, einen Spieler und die Suchtiefe. Die Rückgabewerte sind, der von der Funktion ermittelte, beste Folgezustand und dessen Bewertung. Ermittelt wird diese Zustand durch die Bewertung aller Folgezustände durch die Funktion `value_ab()` und die darauf folgende Auswahl des besten Wertes.

In [ ]:
def best_move_ab(state, player, depth = 5):
    ns     = next_states(state, player)
    moves  = []
    values = []
    for s in ns:
        value = -value_ab(s, opponent(player), depth = depth)
        values = values + [value]
        moves = moves + [s]
    best_value = max(values)
    best_state = moves[values.index(best_value)]
    return best_value, best_state

Die Funktion `alpha_beta_pruning(state, player, depth)` wurde erstellt, um die Funktion `best_move_ab(state, player, depth)` nach außen hin eindeutiger von `best_move_minimax (state, player, depth)` abzugrenzen.

In [ ]:
def alpha_beta_pruning(state, player, depth = 5):
    return(best_move_ab(state, player, depth))

## Funktionstests:

In [ ]:
import time
start = time.time()
state = [[0, 0], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
print(alpha_beta_pruning(state, 2, 7))
end = time.time()
print(str(end-start)+'sec')

In [ ]:
import time
start = time.time()
state = [[0, 0], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
print(minimax(state, 2, 7))
end = time.time()
print(str(end-start)+'sec')

In [ ]:
#import time
#start = time.time()
#state = [[5, 6], [[0, 0, 0, 2, 0, 0, 0, 0], [0, 1, 1, 1, 2, 0, 2, 0], [0, 1, 0, 0, 0, 0, 0, 0]]]
#print(alpha_beta_pruning(state, 2, 5))
#end = time.time()
#print(str(end-start)+'sec')